# Cellpose program. 

This is a program used to generate a new cellpose model based on a series of representative images and their corresponding labelled images. 

In [ ]:
from cellpose import core, models, io, metrics

import os

import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import tqdm 
import tifffile as tf

import tqdm
import tkinter as tk
from tkinter import filedialog


from PIL import Image

import datetime

-------
### Write in cell type

In [ ]:
cell_type = '_Abhi_cells_circs'

### set the Folder path for the test images

In [ ]:
root = tk.Tk()
root.withdraw() # Stops a second window opening
image_folder = filedialog.askdirectory(title = 'Select image Folder')

### Set the folder path for the user defined masks

In [ ]:
root = tk.Tk()
root.withdraw() # Stops a second window opening
mask_folder = filedialog.askdirectory(title = 'Select Masks Folder')

--------------
## Create a method to extract all the filenames from a folder. 

In [ ]:
def get_files_from_folder(folder_path): 
    '''A method to extract all files from the image.'''

    file_list = os.listdir(folder_path)
    image_files = []
    
    for i in range( len(file_list) ): 
        if file_list[i][-4:] == '.tif' or file_list[i][-4:] == '.png':
            image_files.append(file_list[i])
        
    return(image_files)

----
## Create a method to download in the image data from the image file name. 

In [ ]:
def get_image_data(image_file):
    '''Get the image data from the file using Pillow.
    Convert the PILLOW image to a numpy array'''

    image_data = tf.imread(image_file)
    
    # print(image_data.getexif())
    
    np_image_data = np.array(image_data)

    return(np_image_data)

-----
## Collect the image files from folder. 

In [ ]:
image_file_list = get_files_from_folder(image_folder)
mask_file_list = get_files_from_folder(mask_folder)


print(image_folder)
print(mask_folder)
print( len(image_file_list) )
print( len(mask_file_list) )


----
## extract datasets for testing/training

In [ ]:
training_image_index = np.random.choice(len(image_file_list)-1, size = int(0.8*(len(image_file_list)-1)), replace = False)

print('Training image Files :' + str(np.sort(training_image_index)) )
print('Number of training Images: ' + str(len(training_image_index)))

# Create test image index. 
test_image_index = []

for i in range(len(image_file_list)):
    if len(np.where(training_image_index == i)[0]) == 0:
        test_image_index.append(i)

print('Validation image files :' + str(test_image_index) )
print('Number of Validation Images: ' + str(len(test_image_index)) )

--------
## Get test images and user_masks into a format for cellpose model Training. 

In [ ]:
# Initialise 
ground_truth_training = []
training_images = []

for i in range(len(training_image_index)):
    # get the image data
    image_file_name = image_file_list[training_image_index[i]]
    individual_image = get_image_data(image_folder + '/'+ image_file_name)
    training_images.append(individual_image)
    # get the corresponding user_defined_mask
    mask_file_name = image_file_name[0:-4] + '.tif'
    # print(image_file_name)
    # print(mask_file_name)
    user_mask = get_image_data(mask_folder + '/'+ mask_file_name)
    ground_truth_training.append(user_mask)

    
# ground_truth = np.array(ground_truth)
print(training_images[0].shape)
print(ground_truth_training[0].shape)
print(len(training_images))

-----
## Get test images and user_masks into a format for cellpose model evaluation. 

In [ ]:
ground_truth_test = []
test_images = []

for i in range(len(test_image_index)):
    # get the image data
    image_file_name = image_file_list[test_image_index[i]]
    individual_image = get_image_data(image_folder + '/'+ image_file_name)
    test_images.append(individual_image)
    # get the corresponding user_defined_mask
    mask_file_name = image_file_name[0:-4] + '.tif'
    # print(image_file_name)
    # print(mask_file_name)
    user_mask = get_image_data(mask_folder + '/'+ mask_file_name)
    ground_truth_test.append(user_mask)

    
# ground_truth = np.array(ground_truth)
print(test_images[i].shape)
print(ground_truth_test[i].shape)
print(len(test_images))

---------
### Create a meta_data file

In [ ]:
def save_metadata():
    '''Save the training parameters for the deep learning model.  '''

    meta_data = pd.DataFrame({'model_name': [model_name], 
                              'Number of Training Images': [num_training_ims], 
                              'Number of Test Images': [num_testing_ims], 
                              'Number of Epochs': [n_epochs],  
                              'Learning Rate for training': [learning_rate], 
                              'Weight Decay for training': [weight_decay], 
                              'Training data split': [training_data_split], 
                              'Validation data split': [1 - training_data_split], 
                              'Number of images per epoch': [ims_per_epoch], 
                              'Model from the zoo': [start_model], 
                              'Model Accuracy': [ap[:,0].mean()]})
    
    meta_data.to_csv(
                    os.path.dirname(image_folder) + '/models/' + model_name + '_meatadata.csv')

--- 

---
## I will attempt to train a model on the training datasets. 

In [ ]:
# This cell is taken directly from the Colab notebook and then 
# modified for my needs. 

# Get the time and date for the mask name. 
date_time_vals = str(datetime.datetime.now())

date = date_time_vals[0:10].replace('-', '_')

point_find = date_time_vals.find('.')
time = date_time_vals[11:point_find].replace(':', '_')

start_model = 'cyto'

# start logger (to see training across epochs)
logger = io.logger_setup()

# set channels
channels = [0, 0]

n_epochs = 100

learning_rate = 0.1

weight_decay = 0.0001

ims_per_epoch = 8

training_data_split = 0.8

num_training_ims = int(training_data_split * len(training_images)) 

num_testing_ims = len(training_images) - int(training_data_split * len(training_images)) 

model_name =  date + '_' + time + '_' + cell_type 

# DEFINE CELLPOSE MODEL (without size model)
model = models.CellposeModel(gpu=True, model_type=start_model)
# model = openvino_utils.to_openvino(model)

test_files2 = list(test_images)
test_masks2 = list(ground_truth_test)

new_model_path = model.train(training_images[0 : num_training_ims ], ground_truth_training[0 : num_training_ims ], 
                              test_data=training_images[num_training_ims:],
                              test_labels=ground_truth_training[num_training_ims:],
                              channels=channels, 
                              save_path=os.path.dirname(image_folder), 
                              n_epochs=n_epochs,
                              learning_rate=learning_rate, 
                              weight_decay=weight_decay, 
                              nimg_per_epoch=ims_per_epoch,
                              model_name = model_name )

# diameter of labels in training images
diam_labels = model.diam_labels.copy()

print(test_images[0].shape)

In [ ]:
retrained_masks = []

model_path = 'C:/Users/rcorbyn/.cellpose/models/' + '2024_06_05_14_36_50__Abhi_cells_circs'
model_name = '2024_06_05_14_36_50__Abhi_cells_circs'
print(model_path)

# declare model
model = models.CellposeModel(gpu=True, pretrained_model=model_path)

for i in tqdm.tqdm(range(len(test_images) )):
    masks = model.eval(test_images[i], channels = [0, 0], diameter = None)[0]
    retrained_masks.append(masks)

# Check the performance of the model using IoU metric. 
ap = metrics.average_precision(ground_truth_test, retrained_masks)[0]
print(ap[:,0].mean()) 

save_metadata()

In [ ]:
for i in range( len(test_image_index) ):
    print(image_file_list[test_image_index[i]])

In [ ]:
im = 10
fig1, ax1 = plt.subplots()
ax1.imshow(test_images[im][ :, :], vmin = 50)
ax1.imshow(retrained_masks[im], alpha = 0.25, cmap = 'inferno_r', vmax = 1)
ax1.set_title('Cellpose Masks')

fig3, ax3 = plt.subplots()
ax3.imshow(test_images[im][ :, :], vmin = 50)
ax3.imshow(ground_truth_test[im], alpha = 0.25, cmap = 'inferno_r', vmax = 1)
ax3.set_title('Ground truth')

In [ ]:
print(test_images[im].shape)
print(len(retrained_masks))
# %matplotlib
# plt.figure(figsize=(40,16))
fig, ax = plt.subplots()
ax.imshow(test_images[im][ :, :])
fig2, ax2 = plt.subplots()
ax2.imshow(retrained_masks[im], alpha = 0.25, cmap = 'inferno_r', vmax = 1)
# print(np.max(retrained_masks[2]))
